<a href="https://colab.research.google.com/github/julianovale/PO240_Meta_heuristica/blob/main/PO240_Live4_Modelagens_Temas_Anibal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Apresentação fonte dos modelos

https://drive.google.com/file/d/1II4zv1ZBUrEM4ZKfgD35mtopI-uLMLMz/view?usp=sharing

# Modelo do Tema 1: Combustível

1. **Variáveis de decisão**:

$x_{i,j}$ - quantidade de galões (litros) comprado da companhia $i$ ($i = 1, 2, 3$) para o aeroporto $j$ ($j$ = 1 (congonhas), 2 (viracopos), 3 (galeão), 4 (pampulha)).

2. **Parâmetros**:

$c_{i,j}$ - custo de transporte entre a empresa $i$ para o aeroporto $j$.

$f_{i}$ - capacidade da companhia $i$.

$d_{j}$ - demanda do aeroporto $j$.

3. **Restrições**:

3.1. **Da capacidade de fornecimento das companhias**:

$x_{1,1} + x_{1,2} + x_{1,3} + x_{1,4} \leq 250$

$x_{2,1} + x_{2,2} + x_{2,3} + x_{2,4} \leq 500$

$x_{3,1} + x_{3,2} + x_{3,3} + x_{3,4} \leq 600$

3.2. **Do atendimento da demanda dos aeroportos**:

$x_{1,1} + x_{2,1} + x_{3,1} = 100$

$x_{1,2} + x_{2,2} + x_{3,2} = 200$

$x_{1,3} + x_{2,3} + x_{3,3} = 300$

$x_{1,4} + x_{2,4} + x_{3,4} = 400$


4. **Função objetivo**:

$\displaystyle \sum_{i=1}^{M} \sum_{j=1}^{N} c_{i,j} x_{i,j}$



In [ ]:
# importação do pacote para utilização dos solvers: 
# GLOP_LINEAR_PROGRAMMING para problemas de PL ( xi >= 0 )
# CBC para problemas de PLIM ( xi >= 0 e inteiras)
!pip install ortools

In [ ]:
from ortools.linear_solver import pywraplp

# Dados do problema.
# Custos de transporte
costs = [
        [12,  10,  8, 11],
        [ 9,  11, 11, 13],
        [10,  14, 13,  9]
        ]
# Capacidade das companhias
f = [250000, 500000, 600000]
# Demanda dos mercados
d = [100000, 200000, 300000, 400000]

num_companhias = len(costs)
num_mercados = len(costs[0])

# Solver
# Criando uma interface para usar o solver.
solver = pywraplp.Solver('Modelo de Transporte', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
#solver = pywraplp.Solver.CreateSolver('SCIP')

# Variáveis
# x[i, j] é uma matriz of de fluxos da fábrica i
# para o mercado j.
x = {}
for i in range(num_companhias):
  for j in range(num_mercados):
    x[i,j] = solver.NumVar(0, solver.infinity(), '')

# Restrições
# Para cada fábrica a soma dos fluxos de saída
# não pode exceder sua capacidade de produção.
for i in range(num_companhias):
  solver.Add(solver.Sum([x[i, j] for j in range(num_mercados)]) <= f[i],name='RCompanhias '+str(i))

# Para cada mercado a soma dos fluxos de entrada
# deve atender a demanda.
for j in range(num_mercados):
  solver.Add(solver.Sum([x[i, j] for i in range(num_companhias)]) == d[j],name='RMercado '+str(j))

# Função Objetivo.
objective_terms = []
for i in range(num_companhias):
  for j in range(num_mercados):
    objective_terms.append(costs[i][j] * x[i, j])
solver.Minimize(solver.Sum(objective_terms))

# Resolve
status = solver.Solve()

# Relatório da solução.
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print('Custo total = ', solver.Objective().Value(), '\n')
    for i in range(num_companhias):
      cap = 0
      for j in range(num_mercados):
          # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
          if x[i, j].solution_value() > 0.01:
            cap = cap + x[i, j].solution_value()
            print('Fluxo da Companhia %d para o Mercado %d: Fluxo = % 8.2f, Custo = %.2f' %
                  (i+1, j+1, x[i, j].solution_value(), costs[i][j]))
      print('Capacidade utilizada da companhia %d: %8.2f' % (j,cap)) 
      print('---------------------------------------------------- \n')


Custo total =  9250000.0 

Fluxo da Companhia 1 para o Mercado 3: Fluxo =  250000.00, Custo = 8.00
Capacidade utilizada da companhia 3: 250000.00
---------------------------------------------------- 

Fluxo da Companhia 2 para o Mercado 1: Fluxo =  100000.00, Custo = 9.00
Fluxo da Companhia 2 para o Mercado 2: Fluxo =  200000.00, Custo = 11.00
Fluxo da Companhia 2 para o Mercado 3: Fluxo =  50000.00, Custo = 11.00
Capacidade utilizada da companhia 3: 350000.00
---------------------------------------------------- 

Fluxo da Companhia 3 para o Mercado 4: Fluxo =  400000.00, Custo = 9.00
Capacidade utilizada da companhia 3: 400000.00
---------------------------------------------------- 



# Modelo do Tema 2: Refinaria

------------------------------------------------------

1. **Variáveis de decisão**:

------------------------------------------------------

$y_i$ - quantidade de barris comprada da origem $i$,

$x_{i,j}$ -´quantidade da origem $i$ para fazer o produto $j$ (j = 1 (gás), 2 (gasolina), 3 (querosene), 4 (óleo diesel))).

$r_{i,k}$ - quantidade de resíduo para fazer o produto $k$ (1 - óleo pesado, 2 - óleo leve) da origem $i$.

$q_{i,j}$ - quantidade de querosene para fazer o produto $k$ (1 - óleo pesado, 2 - óleo leve) da origem $i$.

$z_k$ - quantidade do óleo $k$ (1 - pesado, 2 - leve). 

------------------------------------------------------

2. **Restrições**:

------------------------------------------------------

2.1 **Restrições entre quantidade de uma origem e seus componentes**:

$y_1 = 0.1*x_{11} + 0.10*x_{12} + 0.10*(x_{13} + q_{11} + q_{12}) + 0.10*x_{14} + 0.60*(r_{11}+r_{12})$

$y_2 = 0.1*x_{21} + 0.15*x_{22} + 0.15*(x_{23} + q_{21} + q_{22}) + 0.15*x_{24} + 0.45*(r_{21}+r_{22})$

$y_3 = 0.1*x_{31} + 0.20*x_{32} + 0.20*(x_{33} + q_{31} + q_{32}) + 0.20*x_{34} + 0.30*(r_{31}+r_{32})$

**Observação**: a quantidade $(r_{1,1} + r_{1,2})$ corresponde a parcela de resíduo da origem i = 1 usada para fazer óleo pesado (1) ou leve (2), respectivamente. A quantidade $(q_{1,1}+q_{1,2})$ corresponde a parcela de querosene da origem i = 1 usada para fazer óleo pesado (1) ou leve (2), respectivamente.  

-------------------------------------------------------

2.2 **Restrição para o total de óleo pesado e leve**:
$z_1 = (q_{11} + q_{21} + q_{31})+(r_{11} + r_{21} + r_{31})$
$z_2 = (q_{12} + q_{22} + q_{32})+(r_{12} + r_{22} + r_{32})$

-------------------------------------------------------

2.3 **Restrições de querosene e resíduo da composição do óleo pesado e leve**:
 
$0.1*z_1 = q_{11}+q_{21}+q_{31}$

$0.9*z_1 = r_{11}+r_{21}+r_{31}$

$0.2*z_2 = q_{12}+q_{22}+q_{32}$

$0.8*z_2 = r_{12}+r_{22}+r_{32}$

-------------------------------------------------------

2.4 **Restrições de limite de compra total por origem**:

$y_1 \leq 70000$

$y_2 \leq 100000$

$y_3 \leq 50000$

-------------------------------------------------------

2.5 **Limites de produção de cada produto**:

$5000  \leq x_{11} + x_{21} + x_{31} \leq 10000$

$13000 \leq x_{12} + x_{22} + x_{32} \leq 20000$

$15000 \leq ((x_{13}+q_{11}+q_{12}) + (x_{23}+q_{21}+q_{22}) + (x_{33}+q_{31}+q_{32})) \leq 20000$

$10000 \leq x_{14} + x_{24} + x_{34} \leq 25000$

$10000 \leq z_{1} \leq 20000$

$12000 \leq z_{2} \leq 20000$

-------------------------------------------------------

3. **Função objetivo**:

------------------------------------------------------

Max $2.1*(x_{11}+x_{21}+x_{31})+3.5*(x_{12}+x_{22}+x_{32})+3.3*(x_{13}+x_{23}+x_{33})
+3.1*(x_{14}+x_{24}+x_{34})+2.5*z_{1}+2.8*z_{2}
-2*y_1-2.5*y_2-3.0*y_3$

------------------------------------------------------


# Modelo do Tema 3: Ligas Metálicas

---

1. **Variáveis de decisão**:

---

$y_i$ - quantidade de minério $i$.  

$x_{i,j}$ - quantidade de metal $j$ obtida a partir do minério $i$.

$z_{j,k}$ - quantidade de metal j na liga $k$ (a = liga A, b = liga B).

$z_k$ - quantidade de liga $k$ (a = liga A, b = liga B).

---

2. **Restrições**:

---

2.1. **Restrições de máxima compra de minérios i**:

$y_1 \leq 1000$

$y_2 \leq 2000$

$y_3 \leq 3000$

---

2.2 **Restrições de % de metais j nos minérios i**:

$y_1 = 0.20*x_{11} + 0.10*x_{12} + 0.30*x_{13} + 0.30*x_{14}$

$y_2 = 0.10*x_{21} + 0.20*x_{22} + 0.30*x_{23} + 0.30*x_{24}$

$y_3 = 0.05*x_{31} + 0.05*x_{32} + 0.70*x_{33} + 0.20*x_{34}$

---

2.3 **Total de metais j e destinação para as ligas k**

$z_{1,a} + z_{1,b} = (x_{11}+x_{21}+x_{31})$

$z_{2,a} + z_{2,b} = (x_{12}+x_{22}+x_{32})$

$z_{3,a} + z_{3,b} = (x_{13}+x_{23}+x_{33})$

$z_{4,a} + z_{4,b} = (x_{14}+x_{24}+x_{34})$

---

2.4 **Total de uma liga k a partir dos metais j**:

$z_a = z_{1,a} + z_{2,a} + z_{3,a} + z_{4,a}$

$z_b = z_{1,b} + z_{2,b} + z_{3,b} + z_{4,b}$ 

---

2.5 **Percentuais mínimos e máximos de metais j em ligas k**:

$z_{1,a} \leq 0.8*z_a$ 

$z_{2,a} \leq 0.3*z_a$

$z_{4,a} \leq 0.5*z_a$ 


$0.4*z_b \leq z_{1,b} \leq 0.6*z_b$ 

$z_{2,b} \geq 0.3*z_b$

$z_{4,b} \leq 0.7*zb$ 

---

3. **Função objetivo**:

---

Max $200*z_a + 300*z_b - 30*y_1 - 40*y_2 - 50*y_3$

---

# Modelo do Tema 4: Ração

---
1. **Variáveis de decisão**:

---

$y_i$ = quantidade do ingrediente i (1 = soja, 2 = milho, 3 = cana).

$x_{i,j}$ = quantidade do nutriente $j$ vindo do ingrediente $i$.

---

2. **Restrições**:

---

2.1 **Total de ração**:

$y_t = 1000$

---

2.2 **Total de ração em função dos ingredientes**:

$y_1 + y_2 + y_3 = y_t$ 

---

2.3 **Quantidade de nutriente j vindo do ingrediente i**:

Soja para cálcio: $0.02*y_1 = x_{1,1}$ 

Soja para proteína: $0.50*y_1 = x_{1,2}$ 

Soja para carbo: $0.08*y_1 = x_{1,3}$

Milho para cálcio: $0.01*y_2 = x_{2,1}$

Milho para proteína: $0,09*y_2 = x_{2,2}$

Milho para carbo: $0.02*y_2 = x_{2,3}$

Cana para cálcio: $0.03*y_3 = x_{3,1}$

Cana para carbo: $0.02*y_3 = x_{3,3}$

---

2.4 **Limite de nutrientes da ração**:

Cálcio: $0.008*y_t \leq x_{1,1} + x_{2,1} + x_{3,1} \leq 0.012*y_t$

Proteína: $0.22*y_t \leq x_{1,2} + x_{2,2}$

Carbo: $x_{1,3} + x_{2,3} + x_{3,3} \leq 0.20*y_t$

---

2.5 **Função objetivo**:

Min $15*y_1 + 20*y_2 + 8*y_3$


